# Deep Learning with Python

## 4.4 Overfitting and underfitting

> 过拟合和欠拟合

机器学习里面有一组问题：**优化**（optimization）和**泛化**（generalization）。

优化不足就是欠拟合，泛化无力即为过拟合。

欠拟合就要接着训练，深度学习网络总是可以优化到一定精度的，关键的问题是泛化。处理过拟合用正则化（regularization），以下是几种正则化的手段：

### 增加训练数据

增加训练集的数据，emmmmm，找更多的数据给它看呀，见多识广啊，这就泛化了。

### 减小网络大小

减小层数和层的单元数，即减小总的参数个数（叫做模型的容量，capacity），这样就可以缓解过拟合。

参数越多，记忆容量越大，就是会死记硬背啦，不利于泛化，题目稍变它心态就崩了。但如果参数太少，记忆容量小，它又学不会、记不住知识，会欠拟合（这知识它不进脑子呀）。

我们要在容量过大和过小之间找个平衡，但这个没有万能公式可以套。要自己多做些尝试，用验证集去评估不同的选择，最终选出最好的来。确定这个的一般方法是：从一个相对较小的值开始，逐步增加，直到对结果基本没有影响了。

### 添加权重正则化

和奥卡姆剃刀原理一致，一个简单模型比一个复杂模型更不容易过拟合。这里说的简单模型是指一个参数取值的分布熵更小的模型。

因此，我们可以强制模型权重取较小的值，从而限制模型的复杂度，来降低过拟合。这种使权重值的分布更加规则(regular)的方法叫作权重正则化
(weight regularization)。具体的实现方法是在损失函数中添加与较大权重值相关的成本。

添加这个成本的方法主要有两种：

- L1 regularization：添加与权重系数的绝对值成正比的值（权重的 L1 范数，the L1 norm of the weights）；
- L2 regularization：添加与权重系数的平方值成正比的值（权重的 L2 范数，the L2 norm of the weights），在神经网络中，L2 正则化也叫权重衰减(weight decay)。

在 Keras 里，添加权重正则化可以通过在层里传一个权重正则化实例来实现：

```python
from keras import regularizers

model = models.Sequential()

model.add(layers.Dense(16, kernel_regularizer=regularizers.l2(0.001), 
                       activation='relu', input_shape=(10000,)))
model.add(layers.Dense(16, kernel_regularizer=regularizers.l2(0.001),
                       activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
```

`regularizers.l2(0.001)` 的意思是在损失函数中添加一项惩罚 `0.001 * weight_coefficient_value`。这个东西只在训练的时候被添加，在用测试集评估的时候不会生效，所以你看到的训练时的损失值要远大于测试时的。

这里也可以把 L2 换成用 L1：`regularizers.l1(0.001)`，或者 L1、L2 同时上：`regularizers.l1_l2(l1=0.001, l2=0.001)`。

### 添加 dropout

对于神经网络的正则化，Dropout 其实才是是最常用、有效的方法。

对某一层使用 dropout，就是在训练过程中随机舍弃一些层输出的特征(就是把值设为 0)。

比如：`[0.2, 0.5, 1.3, 0.8, 1.1]` -> `[0, 0.5, 1.3, 0, 1.1]`，变成 0 的位置是随机的哦。

这里有一个 dropout rate，表示把多少比例的特征置为0，这个比例通常取 0.2~0.5。还有，在测试的时候不 dropout 啊，但测试时的输出要按照 dropout rate 来缩小，以平衡测试和训练时的结果。

也就是说，比如我们搞一个 dropout rate 为 0.5 的，即在训练的时候扔一半：

```python
# training case
layer_output *= np.random.randint(0, high=2, size=layer_output.shape)
```

然后测试的时候，相应的缩小50%：

```python
# testing case
layer_output *= 0.5
```

但通常，我们不是在测试的时候缩小；而是在训练的时候扩大，然后测试的时候就不用动了：

```python
# training case
layer_output *= np.random.randint(0, high=2, size=layer_output.shape)
layer_output /= 0.5
```

![训练时对激活矩阵使用 dropout，并在训练时成比例增大。测试时激活矩阵保持不变](https://tva1.sinaimg.cn/large/007S8ZIlgy1ggl4f43cguj31iy0dc77b.jpg)

在 Keras 里，我们可以通过添加 Dropout 层来实现这个东西：

```python
model.add(layers.Dropout(0.5))
```

例如，我们在我们 IMDB 的网络里面加上 Dropout：

```python
model = models.Sequential()

model.add(layers.Dense(16, activation='relu', input_shape=(10000,)))
model.add(layers.Dropout(0.5))

model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dropout(0.5))

model.add(layers.Dense(1, activation='sigmoid'))
```